In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [87]:
df_train_X = pd.read_csv('train_X.csv')
df_train_y = pd.read_csv('train_y.csv')
df_test_X = pd.read_csv('test1_X.csv')

In [88]:
columns_test = df_test_X.dropna(axis=1).columns
columns_train = df_train_X.dropna(axis=1).columns

In [89]:
for i in (columns_train == columns_test):
    if i != True:
        print(i)

In [90]:
columns_train.shape, columns_test.shape

((107,), (107,))

In [91]:
columns = columns_test

In [92]:
df_test_X = df_test_X[columns]
df_train_X = df_train_X[columns]
(df_train_X.isna().sum() != 0).sum()

np.int64(0)

In [93]:
(df_test_X.isna().sum() != 0).sum()

np.int64(0)

In [46]:
def preprocess_datetime(df):
    # Ensure columns are in datetime format
    df['report_date'] = pd.to_datetime(df['report_date'])
    df['contract_date'] = pd.to_datetime(df['contract_date'])

    # Extract year, month, day, weekday, etc.
    df['report_year'] = df['report_date'].dt.year
    df['report_month'] = df['report_date'].dt.month
    df['report_day'] = df['report_date'].dt.day
    df['report_weekday'] = df['report_date'].dt.weekday

    df['contract_year'] = df['contract_date'].dt.year
    df['contract_month'] = df['contract_date'].dt.month
    df['contract_day'] = df['contract_date'].dt.day
    df['contract_weekday'] = df['contract_date'].dt.weekday

    # Drop original datetime columns if no longer needed
    df = df.drop(columns=['report_date', 'contract_date'])
    
    # print([i for i in df.columns if 'date' in i])
    return df

In [47]:
X_train = preprocess_datetime(df_train_X)
X_test = preprocess_datetime(df_test_X)

In [48]:
X_train.head()

,contract_id,specialization_id,contract_init_sum,project_id,building_id,contractor_id,contract_current_sum,agg_all_contracts__g_contract__bit_da_guid__isMain__count__ALL_TIME,agg_cec_requests__g_contract__request_id__all__count__1W,agg_cec_requests__g_contract__request_id__all__count__2W,...,agg_materials__g_contract__material_type_id__countDistinct__ALL_TIME,agg_materials__g_contract__material_id__countDistinct__ALL_TIME,report_year,report_month,report_day,report_weekday,contract_year,contract_month,contract_day,contract_weekday
0,5433,18,1.115267,43,701,438,0.793952,-0.539030,-0.255766,0.341961,...,5.222766,6.111811,2023,1,1,6,2022,5,10,1
1,6875,18,1.608002,31,268,438,2.681675,0.743673,0.808046,0.662319,...,3.387956,5.004645,2023,1,1,6,2022,8,21,6
2,1476,18,-0.360764,31,268,438,-0.416432,-0.539030,-0.255766,-0.298754,...,-0.281663,0.308738,2023,1,1,6,2022,10,17,0
3,4469,12,-0.089303,43,697,484,-0.193827,-0.539030,-0.255766,0.021603,...,-0.159342,-0.187578,2023,1,1,6,2022,4,20,2
4,1330,12,-0.515778,49,224,484,0.133446,0.743673,-0.255766,-0.298754,...,-0.159342,-0.034865,2023,1,1,6,2022,3,2,2


In [49]:
X_test.head()

,contract_id,specialization_id,contract_init_sum,project_id,building_id,contractor_id,contract_current_sum,agg_all_contracts__g_contract__bit_da_guid__isMain__count__ALL_TIME,agg_cec_requests__g_contract__request_id__all__count__1W,agg_cec_requests__g_contract__request_id__all__count__2W,...,agg_materials__g_contract__material_type_id__countDistinct__ALL_TIME,agg_materials__g_contract__material_id__countDistinct__ALL_TIME,report_year,report_month,report_day,report_weekday,contract_year,contract_month,contract_day,contract_weekday
0,3029,15,-0.347029,42,84,637,-0.405169,-0.53903,-0.255766,-0.298754,...,-0.403984,-0.34029,2023,7,30,6,2023,7,24,0
1,4350,11,-0.092977,44,727,637,-0.196840,-0.53903,-0.255766,-0.298754,...,-0.403984,-0.34029,2023,7,30,6,2023,7,11,1
2,1095,15,-0.467957,44,728,637,-0.504333,-0.53903,-0.255766,-0.298754,...,-0.403984,-0.34029,2023,7,30,6,2023,7,24,0
3,2634,15,-0.486723,44,727,637,-0.519721,-0.53903,-0.255766,-0.298754,...,-0.403984,-0.34029,2023,7,30,6,2023,7,24,0
4,6535,11,-0.452585,42,85,637,-0.491727,-0.53903,-0.255766,-0.298754,...,-0.403984,-0.34029,2023,7,30,6,2023,7,27,3


In [50]:
df_train_y

,contract_id,report_date,default6
0,5433,2023-01-01,0
1,6875,2023-01-01,0
2,1476,2023-01-01,0
3,4469,2023-01-01,0
4,1330,2023-01-01,0
...,...,...,...
28826,5078,2023-07-23,0
28827,3854,2023-07-23,0
28828,5351,2023-07-23,0
28829,57,2023-07-23,0


In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

train_X, test_X, train_y, test_y = train_test_split(X_train, df_train_y.default6, test_size=0.2, random_state=42)

# Define the XGBoost model
xgboost = XGBClassifier(use_label_encoder=True, eval_metric='aucpr')

# Define the parameter grid for GridSearchCV
parameters = {
    'n_estimators': [500],
    'learning_rate': [0.1],
    'max_depth': [7],
    'min_child_weight': [1],
    'gamma': [0],
    'subsample': [0.8],
    'colsample_bytree': [0.8]
}

# Set up the GridSearchCV
clf = GridSearchCV(estimator=xgboost, param_grid=parameters, scoring='f1', cv=5, n_jobs=-1)

# Fit the model
clf.fit(train_X, train_y)

# Print the best parameters and best score
print("Best Parameters:", clf.best_params_)
print("Best CV Score:", clf.best_score_)

# Make predictions on the test set
y_pred = clf.predict(test_X)

# Evaluate the model
print("Test Set Accuracy:", accuracy_score(test_y, y_pred))
print("\nClassification Report:\n", classification_report(test_y, y_pred))

/home/clodis/Projects/Python/Plane_aichal/.conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [13:58:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/clodis/Projects/Python/Plane_aichal/.conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [13:58:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/clodis/Projects/Python/Plane_aichal/.conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [13:58:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/clodis/Projects/Python/Plane_aichal/.conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [13:58:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Parameters: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 500, 'subsample': 0.8}
Best CV Score: 0.9715446551516024
Test Set Accuracy: 0.9932373851222472

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4848
           1       0.98      0.97      0.98       919

    accuracy                           0.99      5767
   macro avg       0.99      0.99      0.99      5767
weighted avg       0.99      0.99      0.99      5767



In [94]:
from sklearn .metrics import roc_auc_score
auc = roc_auc_score(test_y, y_pred)
auc * 2 - 1

np.float64(0.9716724664849512)

In [95]:
ans = pd.read_csv('submission.csv')
# ans.score = clf.predict(X_test)

In [96]:
print(df_test_X[['contract_id', 'report_date']].dtypes)
print(ans[['contract_id', 'report_date']].dtypes)


contract_id     int64
report_date    object
dtype: object
contract_id     int64
report_date    object
dtype: object


In [146]:
res = pd.merge(df_test_X[columns], ans, how='inner', on = ['contract_id', 'report_date'])
res2 = preprocess_datetime(res)
res2.score = clf.predict_proba(res2.drop(columns='score'))

In [147]:
res2.score

0       0.997644
1       0.988558
2       0.960171
3       0.990984
4       0.999268
          ...   
7327    0.998239
7328    0.947660
7329    0.893537
7330    0.998095
7331    0.986328
Name: score, Length: 7332, dtype: float32

In [125]:
#res2.score.value_counts()[0] + res2.score.value_counts()[1] == res2.shape[0]

In [148]:
res2.score.value_counts()

score
0.999384    3
0.999383    3
0.999584    2
0.996050    2
0.999961    2
           ..
0.818409    1
0.995053    1
0.999450    1
0.999732    1
0.996927    1
Name: count, Length: 7203, dtype: int64

In [149]:
res2

,contract_id,specialization_id,contract_init_sum,project_id,building_id,contractor_id,contract_current_sum,agg_all_contracts__g_contract__bit_da_guid__isMain__count__ALL_TIME,agg_cec_requests__g_contract__request_id__all__count__1W,agg_cec_requests__g_contract__request_id__all__count__2W,...,agg_materials__g_contract__material_id__countDistinct__ALL_TIME,score,report_year,report_month,report_day,report_weekday,contract_year,contract_month,contract_day,contract_weekday
0,3029,15,-0.347029,42,84,637,-0.405169,-0.539030,-0.255766,-0.298754,...,-0.340290,0.997644,2023,7,30,6,2023,7,24,0
1,4350,11,-0.092977,44,727,637,-0.196840,-0.539030,-0.255766,-0.298754,...,-0.340290,0.988558,2023,7,30,6,2023,7,11,1
2,1095,15,-0.467957,44,728,637,-0.504333,-0.539030,-0.255766,-0.298754,...,-0.340290,0.960171,2023,7,30,6,2023,7,24,0
3,2634,15,-0.486723,44,727,637,-0.519721,-0.539030,-0.255766,-0.298754,...,-0.340290,0.990984,2023,7,30,6,2023,7,24,0
4,6535,11,-0.452585,42,85,637,-0.491727,-0.539030,-0.255766,-0.298754,...,-0.340290,0.999268,2023,7,30,6,2023,7,27,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7327,4740,22,-0.505882,26,167,83,-0.535433,-0.539030,-0.255766,-0.298754,...,-0.340290,0.998239,2023,9,24,6,2023,8,23,2
7328,2689,9,0.071884,56,657,396,-0.061649,-0.539030,-0.255766,-0.298754,...,-0.225756,0.947660,2023,9,24,6,2023,2,27,0
7329,7135,14,-0.515778,47,608,580,-0.508195,0.743673,-0.255766,-0.298754,...,0.499628,0.893537,2023,9,24,6,2023,5,16,1
7330,4164,14,-0.344397,49,241,582,-0.403011,-0.539030,-0.255766,-0.298754,...,-0.340290,0.998095,2023,9,24,6,2023,8,5,5


In [150]:
(ans.contract_id == res2.contract_id).sum() # Всё ок порядок сохранён
ans.score = res2.score
ans.score.unique()

array([0.99764395, 0.9885579 , 0.9601705 , ..., 0.89353746, 0.998095  ,
       0.98632795], dtype=float32)

In [154]:
ans.score = 1 - ans.score
ans.to_csv('submission.csv', index = False)

In [152]:
ans

,contract_id,report_date,score
0,3029,2023-07-30,0.997644
1,4350,2023-07-30,0.988558
2,1095,2023-07-30,0.960171
3,2634,2023-07-30,0.990984
4,6535,2023-07-30,0.999268
...,...,...,...
7327,4740,2023-09-24,0.998239
7328,2689,2023-09-24,0.947660
7329,7135,2023-09-24,0.893537
7330,4164,2023-09-24,0.998095


In [83]:
df_test_X.report_date
ans.report_date
df_test_X.contract_id
ans.contract_id

0       3029
1       4350
2       1095
3       2634
4       6535
        ... 
7327    4740
7328    2689
7329    7135
7330    4164
7331     194
Name: contract_id, Length: 7332, dtype: int64

In [145]:
ans.max()

contract_id          7338
report_date    2023-09-24
score             0.99069
dtype: object

In [144]:
clf.predict_proba(res2.drop(columns='score'))

array([[0.99764395, 0.00235607],
       [0.9885579 , 0.01144213],
       [0.9601705 , 0.03982949],
       ...,
       [0.89353746, 0.10646253],
       [0.998095  , 0.001905  ],
       [0.98632795, 0.01367203]], dtype=float32)